In [ ]:
import pandas as pd
import numpy as np
import tensorflow
import cv2 
import os

In [ ]:
TRAIN_DIR = '/datasets/louis/train'
IMG_SIZE = 224

In [ ]:
import sys
sys.path.append('../..')
from invasive_species.src.process_data import process_data

In [ ]:
x_train, img_nums = process_data(TRAIN_DIR, 50, IMG_SIZE)

In [ ]:
labels = pd.read_csv('../data/train_labels.csv')

In [ ]:
len(x_train)

50

In [ ]:
y_train = []
for n in img_nums:
    label = labels[labels['name']==int(n)]['invasive']
    y_train.append(label.iloc[0])

In [ ]:
len(y_train)

50

# sckit Learn

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='adam', max_iter=5, random_state=0)

In [ ]:
x_train_np = np.array(x_train)

In [ ]:
nb, x, y = x_train_np.shape
x_train_2d = x_train_np.reshape(nb, x*y)

In [ ]:
clf.fit(x_train_2d, y_train)

/opt/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(max_iter=5, random_state=0)

In [ ]:
import pickle
filename = 'model_sklearn_mlpclassifier.pkl'
pickle.dump(clf, open('../data/'+filename, 'wb'))

# AutoKeras

In [ ]:
import autokeras as ak

In [ ]:
clf = ak.ImageClassifier()

INFO:tensorflow:Reloading Oracle from existing project ./image_classifier/oracle.json
INFO:tensorflow:Reloading Tuner from ./image_classifier/tuner0.json


In [ ]:
y_train = []
for n in img_nums:
    label = labels[labels['name']==int(n)]['invasive']
    y_train.append(label.iloc[0])

In [ ]:
clf.fit(np.array(x_train), np.array(y_train), epochs=1)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
image_block_1/b...|efficient         |vanilla           
image_block_1/n...|True              |True              
image_block_1/a...|True              |False             
image_block_1/i...|True              |None              
image_block_1/i...|False             |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0                 |None              
image_block_1/i...|0.1               |None              
image_block_1/i...|0                 |None              
image_block_1/e...|True              |None              
image_block_1/e...|b7                |None              
image_block_1/e...|True              |None              
image_block_1/e...|True              |None              
classification_...|global_avg        |flatten           
classification_...|0                 |0.5               
optimizer         |adam              |adam              
lear

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
model = clf.export_model()

In [ ]:
try:
    model.save("model_autokeras", save_format="tf")
except:
    model.save("model_autokeras.h5")

In [ ]:
#y_test_pre = clf.predict(x_test)